In [1]:
#import libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import pickle
import tensorflow-gpu as tf


SyntaxError: invalid syntax (2067127434.py, line 12)

In [113]:
#call dataset
spamdata = pd.read_csv("spam.csv",encoding='latin-1')

spamdata.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [114]:
data = spamdata.copy()
data.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], inplace=True)
data = data.rename(columns={"v1":"label", "v2":"text"})

In [115]:
data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [116]:
data['label'] = data['label'].map( {'spam': 1, 'ham': 0} )
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [117]:
data_ham  = data[data['label'] == 0].copy()
data_spam = data[data['label'] == 1].copy()

In [118]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X = data['text'].values
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [119]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(X_train)

# integer encode the documents
encoded_train = t.texts_to_sequences(X_train)
encoded_test = t.texts_to_sequences(X_test)

vocab_size = len(t.word_index) + 1

print(encoded_train[0:2])

[[38, 30, 8, 5, 273, 1989, 81, 116, 26, 11, 1656, 322, 10, 53, 18, 299, 30, 349, 1990], [799, 15, 2555, 1442, 1127, 192, 2556, 171, 12, 98, 1991, 44, 195, 1657, 2557, 1992, 2558, 21, 9, 4, 203, 1025, 225]]


In [120]:
# pad documents to a max length of 4 words
max_length = 8
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

print(padded_train)

[[ 322   10   53 ...   30  349 1990]
 [1992 2558   21 ...  203 1025  225]
 [  83 1443    4 ...    2 3794 3795]
 ...
 [1477   30 2063 ...  239   30 2064]
 [ 763 1679 1161 ...    0    0    0]
 [   8  155   20 ...    8  290  175]]


In [121]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 24, input_length=max_length))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 8, 24)             190920    
                                                                 
 flatten_2 (Flatten)         (None, 192)               0         
                                                                 
 dense_10 (Dense)            (None, 500)               96500     
                                                                 
 dense_11 (Dense)            (None, 200)               100200    
                                                                 
 dense_12 (Dense)            (None, 100)               20100     
                                                                 
 dense_13 (Dense)            (None, 1)                 101       
                                                                 
Total params: 407,821
Trainable params: 407,821
Non-tr

In [122]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# fit the model
model.fit(x=padded_train, 
          y=y_train, 
          epochs=50,
          validation_data=(padded_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

Epoch 1/50
140/140 [==============================] - 1s 5ms/step - loss: 0.1903 - accuracy: 0.9262 - val_loss: 0.1058 - val_accuracy: 0.9731
Epoch 2/50
140/140 [==============================] - 1s 4ms/step - loss: 0.0497 - accuracy: 0.9836 - val_loss: 0.0865 - val_accuracy: 0.9767
Epoch 3/50
140/140 [==============================] - 1s 4ms/step - loss: 0.0176 - accuracy: 0.9942 - val_loss: 0.0982 - val_accuracy: 0.9785
Epoch 4/50
140/140 [==============================] - 1s 4ms/step - loss: 0.0028 - accuracy: 0.9993 - val_loss: 0.1941 - val_accuracy: 0.9776
Epoch 5/50
140/140 [==============================] - 1s 4ms/step - loss: 8.0790e-04 - accuracy: 0.9998 - val_loss: 0.2360 - val_accuracy: 0.9758
Epoch 6/50
140/140 [==============================] - 1s 4ms/step - loss: 2.8087e-07 - accuracy: 1.0000 - val_loss: 0.2970 - val_accuracy: 0.9776
Epoch 7/50
140/140 [==============================] - 1s 4ms/step - loss: 1.9464e-09 - accuracy: 1.0000 - val_loss: 0.3118 - val_accuracy: 0

In [126]:
# evaluate the model
loss, accuracy = model.evaluate(padded_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 97.488791


In [130]:
model.save("spam_model")

INFO:tensorflow:Assets written to: spam_model\assets


In [131]:
with open('spam_model/tokenizer.pkl', 'wb') as output:
    pickle.dump(t, output, pickle.HIGHEST_PROTOCOL)

In [132]:
s_model = tf.keras.models.load_model("spam_model")

with open('spam_model/tokenizer.pkl', 'rb') as input:
    tokener = pickle.load(input)

In [134]:
sms_test = ["Feel Yourself That You Are Always Happy.. Slowly It Becomes Your Habit &amp; Finally It Becomes Part Of Your Life.. Follow It.. Happy Morning &amp; Have A Happy Day:)"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 24ms/step


0

In [135]:
sms_test = ["staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 21ms/step


0

In [137]:
sms_test = ["Send me yetty's number pls."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 20ms/step


0

In [138]:
sms_test = ["Hey so this sat are we going for the intro pilates only? Or the kickboxing too? "]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 31ms/step


0

In [139]:
sms_test = ["I got it before the new year cos yetunde said she wanted to surprise you with it but when i didnt see money i returned it mid january before the  &lt;#&gt; day return period ended."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 25ms/step


0

In [140]:
sms_test = ["Hey we can go jazz power yoga hip hop kb and yogasana "]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 29ms/step


0

In [141]:
sms_test = ["Hey mate. Spoke to the mag people. We‰Û÷re on.  the is deliver by the end of the month. Deliver on the 24th sept. Talk later. "]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 27ms/step


0

In [142]:
sms_test = ["Morning only i can ok."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 24ms/step


0

In [143]:
sms_test = ["Wat time Ì_ finish?"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 27ms/step


0

In [144]:
sms_test = ["Shant disturb u anymore... Jia you..."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 30ms/step


0

In [145]:
sms_test = ["S4mths half price Orange line rental & latest camera phones 4 FREE. Had your phone 11mths ? Call MobilesDirect free on 08000938767 to update now! or2stoptxt"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 33ms/step


1

In [146]:
sms_test = ["Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply.."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 28ms/step


0

In [147]:
sms_test = ["MOON has come to color your dreams, STARS to make them musical and my SMS to give you warm and Peaceful Sleep. Good Night"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 29ms/step


0

In [148]:
sms_test = ["You are a winner U have been specially selected 2 receive å£1000 cash or a 4* holiday (flights inc) speak to a live operator 2 claim 0871277810810"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 28ms/step


1

In [150]:
sms_test = ["Unless it's a situation where YOU GO GURL would be more appropriate"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred


1/1 [==============================] - 0s 23ms/step


0

In [151]:
sms_test = ["www dat is sweet! We can think of something to do he he! Have a nice time tonight ill probably txt u later cos im lonely :( xxx."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred


1/1 [==============================] - 0s 28ms/step


0

In [152]:
sms_test = ["Bring tat cd don forget"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred


1/1 [==============================] - 0s 30ms/step


0

In [153]:
sms_test = ["Single line with a big meaning::::: \Miss anything 4 ur \"Best Life\" but"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred


1/1 [==============================] - 0s 29ms/step


0

In [154]:
sms_test = ["Jay's getting really impatient and belligerent"]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 31ms/step


0

In [155]:
sms_test = ["Lol they were mad at first but then they woke up and gave in."]


sms_proc = tokener.texts_to_sequences(sms_test)
sms_proc = pad_sequences(sms_proc, maxlen=max_length, padding='post')

pred = (model.predict(sms_proc) > 0.5).astype("int32").item()
pred

1/1 [==============================] - 0s 30ms/step


0